In [ ]:
import numpy as np
import cv2
import os
import time
import matplotlib.pyplot as plt
import csv
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

start_time = time.time()
surf = cv2.xfeatures2d.SURF_create()
clf = RandomForestClassifier(n_estimators=100)

# clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
# clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
path = "../X_Train"
count = 0

# #---------------------------------------------#
# #-------------Preprocessing-------------------#
# #---------------------------------------------#
all_image = []
imglist = []

for root, dirs, files in os.walk(path):
	for image in files:
		all_image.append(int(image.split('.')[0]))
all_image = sorted(all_image)

for i in range(0, len(all_image)):
	imglist.append(str(all_image[i]) +".jpg")
# print(len(imglist))


total_img = 4100
length_test = len(imglist) - total_img - 1
# length_test = 50
num_descriptor = 30

# #---------------------------------------------#
# #-------------Populate x array----------------#
# #---------------------------------------------#
count = 0
x_list = np.array( [], dtype="float32")
for image in imglist:

	inner_path = path + "/" +  image
	img = cv2.imread(inner_path)
	keypoints, descriptors = surf.detectAndCompute(img, None)
	x_append = np.reshape(descriptors[:num_descriptor], (1,-1))
	x_append = np.squeeze(x_append)
	# print(x_append.shape)
	x_list = np.append(x_list,x_append)
	count = count + 1
	# print(count)

	if (count >= total_img):
		break

x_list= np.reshape(x_list,(total_img,-1))
x = x_list

# #---------------------------------------------#
# #-------------Populate y array----------------#
# #---------------------------------------------#
y_list = []
with open('../Y_Train.csv') as csvfile:
	reader = csv.DictReader(csvfile)
	for row in reader:
		y_append = (row['Label'])
		y_list.append(y_append)
y = np.asarray(y_list)


# #---------------------------------------------#
# #----------Training the data using SVM--------#
# #---------------------------------------------#
clf.fit(x, y[:total_img]) 

predicted_output = []
correct_output = []

# #---------------------------------------------#
# #-----Testing: Populate Correct Output--------#
# #---------------------------------------------#

count = 0
with open('../Y_Train.csv') as csvfile:
	reader = csv.DictReader(csvfile)
	for row in reader:
		if (int(row['Image'].split('.')[0]) > total_img):
			correct_y = (row['Label'])
			correct_output.append(correct_y)
			count = count + 1
		if (count > length_test):
			break
correct_output = np.asarray(correct_output)


# #---------------------------------------------#
# #--------Predicting the output----------------#
# #---------------------------------------------#
count = 0
for image in imglist:
	if (int(image.split('.')[0]) > total_img):
		inner_path = path + "/" +  image
		img = cv2.imread(inner_path)
		keypoints, descriptors = surf.detectAndCompute(img, None)
		p = np.reshape(descriptors[:num_descriptor], (1,-1))
		p_predict = clf.predict(p)
		predicted_output.append(np.asscalar(p_predict))
		count = count + 1

	if (count > length_test):
		break

predicted_output = np.asarray(predicted_output)

# #---------------------------------------------#
# #-------------Print Accuracy------------------#
# #---------------------------------------------#
print(accuracy_score(correct_output, predicted_output))
print("--- %s Total Time Taken ---" % (time.time() - start_time))